# Getting Started: training an image classification model

**Learning Objectives** - By the end of this quickstart tutorial, you'll know how to train and deploy an image classification model on Azure Machine Learning studio.

This tutorial covers:

- Connect to workspace & set up a compute resource on the Azure Machine Learning Studio Notebook UI
- Bring data in and prepare it to be used for training
- Train a model for image classification
- Metrics for optimizing your model
- Deploy the model online & test

### 1. Connect to Azure Machine Learning workspace

Before we dive in the code, you'll need to connect to your workspace. The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning.

We are using `DefaultAzureCredential` to get access to workspace. `DefaultAzureCredential` should be capable of handling most scenarios. If you want to learn more about other available credentials, go to [set up authentication doc](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk), [azure-identity reference doc](https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

**Make sure to enter your workspace credentials before you run the script below.**

In [1]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

# Get a handle to the workspace. You can find the info on the workspace tab on ml.azure.com
ml_client = MLClient(
    credential=credential,
    subscription_id="xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",  # this will look like xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
    resource_group_name="DefaultResourceGroup-EUS2",
    workspace_name="chest-xrays-automl",
)

### 2. Create compute

In order to train a model on the Notebook editor on Azure Machine Learning studio, you will need to create a compute resource first. This is easily handled through a compute creation wizard. **Creating a compute will take 3-4 minutes.**

![](media/compute-creation.png)

1. Click **...** menu button on the top of Notebook UI, and select **+Create Azure ML Compute Instance**.
2. **Name** the compute as **cpu-cluster**
3. Select **CPU** and **STANDARD_DS3_V2**. 
4. Click **Create**

If you are interested in learning how to create compute via code, see [Azure Machine Learning in a Day](https://github.com/Azure/azureml-examples/blob/main/tutorials/azureml-in-a-day/azureml-in-a-day.ipynb). 

### 3. Create a job environment
To run an Azure Machine Learning training job, you'll need an environment.

In this tutorial, you'll using a ready-made environment called `AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest` that contains all required libraries (python, MLflow, numpy, pip, etc). 

### 4. Build the command job to train
Now that you have all assets required to run your job, it's time to build the job itself, using the Azure ML Python SDK v2. We will be creating a command job.

An AzureML command job is a resource that specifies all the details needed to execute your training code in the cloud: inputs and outputs, the type of hardware to use, software to install, and how to run your code. the command job contains information to execute a single command.

**Create training script**

Let's start by creating the training script - the *main.py* python file.

In [2]:
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

This script handles the preprocessing of the data, splitting it into test and train data. It then consumes this data to train a tree based model and return the output model. [MLFlow](https://mlflow.org/docs/latest/tracking.html) will be used to log the parameters and metrics during our pipeline run.

In [6]:
%%writefile {train_src_dir}/main.py

import os
import argparse
import mlflow
import torchvision
from torchvision import transforms
import torch

def train_model(model,criterion,optimizer,loader,n_epochs,device):
    
    loss_over_time = [] # to track the loss as the network trains
    
    model = model.to(device) # Send model to GPU if available
    model.train() # Set the model to training mode
    
    for epoch in range(n_epochs):  # loop over the dataset multiple times
        
        running_loss = 0.0
        running_corrects = 0
        
        for i, data in enumerate(loader):
            # Get the input images and labels, and send to GPU if available
            inputs, labels = data[0].to(device), data[1].to(device)
            # Convert to one channel image (grayscale)
            inputs = inputs[:,0,:,:].unsqueeze(1)

            # Zero the weight gradients
            optimizer.zero_grad()

            # Forward pass to get outputs
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backpropagation to get the gradients with respect to each weight
            loss.backward()

            # Update the weights
            optimizer.step()

            # Convert loss into a scalar and add it to running_loss
            running_loss += loss.item()

            # Convert loss into a scalar and add it to running_loss
            running_loss += loss.item() * inputs.size(0)
            # Track number of correct predictions
            running_corrects += torch.sum(preds == labels.data)
            
        # Calculate and display average loss and accuracy for the epoch
        epoch_loss = running_loss / len(loader)
        epoch_acc = running_corrects.double() / len(loader)
        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        loss_over_time.append(epoch_loss)

    return loss_over_time

def main():
    """Main function of the script."""
    print('Starting...')
    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--registered_model_name", type=str, help="model name")
    args = parser.parse_args()
   
    # start Logging
    mlflow.start_run()

    ###################
    #<prepare the data>
    ###################
    train_path = os.path.join(args.data, 'train')
    test_path = os.path.join(args.data, 'test')
    
    # Create datasets    
    train_data = torchvision.datasets.ImageFolder(train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize(mean=[0.5],std=[0.5])]))
    test_data = torchvision.datasets.ImageFolder(test_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),transforms.Normalize(mean=[0.5],std=[0.5])]))
    
    # Create dataloader
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)
    
    ###################
    #</prepare the data>
    ###################

    ##################
    #<train the model>
    ##################
    
    model = torchvision.models.resnet18(pretrained=True)
    
    for param in model.parameters():
        param.requires_grad = False
        
    # Replace the resnet input layer to take in grayscale images (1 input channel), since it was originally trained on color (3 input channels)
    in_channels = 1
    model.conv1 = torch.nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

    # Replace the resnet final layer with a new fully connected Linear layer we will train on our task
    # Number of out units is number of classes (3)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, 3)
    
    # Set device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Train the model
    n_epochs = 10
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    cost_path_train = train_model(model,criterion,optimizer,train_dataloader,n_epochs,device)
    
    # Test the model
    cost_path_test = train_model(model,criterion,optimizer,test_dataloader,n_epochs,device)
    
    # Print the train and test loss
    print(f"Train loss: {cost_path_train[-1]}")
    print(f"Test loss: {cost_path_test[-1]}")
    
    ##################
    #</train the model>
    ##################

    ##########################
    #<save and register model>
    ##########################
    # registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.pytorch.log_model(model, args.registered_model_name)

    # saving the model to a file
    mlflow.pytorch.save_model(model, path='latest_model')
    
    # stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()


Overwriting ./src/main.py


As you can see in this script, once the model is trained, the model file is saved and registered to the workspace. Now you can use the registered model in inferencing endpoints.

**Configure the Command**

Now that you have a script that can perform the desired tasks, You'll use the general purpose command that can run command line actions. This command line action can be directly calling system commands or running a script.

Here, you'll use input data, split ratio, learning rate and registered model name as input variables.

In [7]:
# import the libraries
from azure.ai.ml import command, Input

# name the model you registered earlier in the training script
registered_model_name = "chest_xray_classifier"

# configure the command job
job = command(
    inputs=dict(
        # uri_file refers to a specific file as a data asset
        data='chest_xrays',  
        registered_model_name=registered_model_name,  # input variable in main.py
    ),
    code="./",  # location of source code
    # The inputs/outputs are accessible in the command via the ${{ ... }} notation
    command="python src/main.py --data ${{inputs.data}}  --registered_model_name ${{inputs.registered_model_name}}",
    # This is the ready-made environment you are using
    #environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    environment="AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu@latest",
    # This is the compute you created earlier
    compute="chest-xray-compute",
    # An experiment is a container for all the iterations one does on a certain project. All the jobs submitted under the same experiment name would be listed next to each other in Azure ML studio.
    experiment_name="automated_training_experiment",
    display_name="automated_training_run",
)

### 5. Submit the job ###
It's now time to submit the job to run in AzureML. **The job will take 2 to 3 minutes to run**. It could take longer (up to 10 minutes) if the compute instance has been scaled down to zero nodes and custom environment is still building.

In [8]:
# submit the command job
ml_client.create_or_update(job)

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'C:\Duke\AIPI561\Projects\Individual_Project_2\AutoML' 'https://chestxraysauto8122485356.blob.core.windows.net/c5e88b7e-1-e7a43580-b93b-5e87-9420-302122b7802d/AutoML' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading AutoML (332.25 MBs): 100%|##########| 332249803/332249803 [03:47<00:00, 1461324.80it/s]




Experiment,Name,Type,Status,Details Page
automated_training_experiment,frosty_map_58vtrbz7kd,command,Starting,Link to Azure Machine Learning studio


### 6. View the result of a training job

![](media/view-job.gif)


You can view the result of a training job by **clicking the URL generated after submitting a job**. Alternatively, you can also click **Jobs** on the left navigation menu. A job is a grouping of many runs from a specified script or piece of code. Information for the run is stored under that job. 

1. **Overview** is where you can see the status of the job.  
2. **Metrics** would display different visualizations of the metrics you specified in the script.
3. **Images** is where you can view any image artifacts that you have logged with MLflow.
4. **Child jobs** contains child jobs if you added them.
5. **Outputs + logs** contains log files you need for troubleshooting or other monitoring purposes. 
6. **Code** contains the script/code used in the job.
7. **Explanations** and **Fairness** are used to see how your model performs against responsible AI standards. They are currently preview features and require additional package installations.
8. **Monitoring** is where you can view metrics for the performance of compute resources.  


### 7. Deploy the model as an online endpoint

Now deploy your machine learning model as a web service in the Azure cloud, an [`online endpoint`](https://docs.microsoft.com/azure/machine-learning/concept-endpoints).

To deploy a machine learning service, you usually need:

- The model assets (file, metadata) that you want to deploy. You've already registered these assets via MLflow in *main.py*. You can find it under **Models** on the left navigation menu on Azure Machine Learning studio. 
- The code that executes the model on a given input request. In this quickstart, you can easily set it up through the endpoint creation UI. If you want to learn more about how to deploy via Azure Machine Learning SDK, see [Azure Machine Learning in a Day](https://github.com/Azure/azureml-examples/blob/main/tutorials/azureml-in-a-day/azureml-in-a-day.ipynb).

![](media/endpoint-creation.gif)

**Find the endpoint creation wizard on Studio**
1. Open a duplicate tab (so that you can keep this tutorial open).
1. On the duplicate tab, select **Endpoints** on the left navigation menu.
2. Select **+Create** for real-time endpoints.

**Endpoint creation & deployment via wizard UI** (this will take approximately 6 to 8 minutes)
1. Enter a **unique name** for *endpoint name*. We recommend creating a *unique* name with current date/time to avoid conflicts, which could prevent your deployment. Keep all the defaults for the rest. 
2. Next, you need to choose a model to deploy. Select **credit_defaults_model** registered by *main.py* earlier. 
3. Keep all the defaults for deployment configuration.
1. Select **Standard_DS3_V2** for compute, which is what we configured earlier. Set the instance count to **1**.
1. Keep all the defaults for the traffic.
1. Review: review and select **Create**.  

![](media/endpoint-test.gif)

**Test with a sample query**
1. Select the endpoint you just created. Make sure the endpoint is created and the model has been deployed to it.
2. Select the **Test** tab.
3. Copy & paste the following sample request file into the **Input data to test real-time endpoint** field.
4. Select **Test**. 

```
{
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0, 1],
    "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
        ]
  }
}
```

**Clean up resources**

If you're not going to use the endpoint, delete it to stop using the resource. Make sure no other deployments are using an endpoint before you delete it.

1. Click **Details** on the endpoint page.
2. Click the **Delete** button.

**Expect this step to take approximately 6 to 8 minutes.**